# Example of reading / saving data from / to GEFF

This example illustrates how we can save the tracked data into a [GEFF (Graph Exchange File Format)](https://github.com/live-image-tracking-tools/geff).

The GEFF file can be visualized in `napari` by [napari-geff](https://github.com/live-image-tracking-tools/napari-geff).

In [8]:
from laptrack import LapTrack
from laptrack import data_conversion
from laptrack import datasets
import geff
import tempfile
from skimage.measure import regionprops_table
import pandas as pd
from pathlib import Path
import warnings
import shutil

warnings.filterwarnings("ignore", category=UserWarning)
tmp_path = Path(tempfile.gettempdir())
print(f"Temporary files will be stored in: {tmp_path}")

Temporary files will be stored in: /var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T


## Prepare data by centroid tracking

### Calculate regionprops

In [2]:
image, labels = datasets.cell_segmentation()
regionprops = []
for frame, label in enumerate(labels):
    df = pd.DataFrame(regionprops_table(label, properties=["label", "centroid"]))
    df["frame"] = frame
    regionprops.append(df)
regionprops_df = pd.concat(regionprops).rename(
    columns={"centroid-0": "y", "centroid-1": "x", "label": "seg_id"}
)
coordinate_cols = ["y", "x"]
frame_col = "frame"
regionprops_df.head()

,seg_id,y,x,frame
0,1,137.033613,96.336134,0
1,2,145.068966,178.988506,0
2,3,123.091743,203.266055,0
3,4,62.716216,141.290541,0
4,5,32.976744,124.348837,0


### Tracking with LapTrack

In [3]:
lt = LapTrack(
    metric="sqeuclidean",
    cutoff=15**2,
    splitting_metric="sqeuclidean",
    merging_metric="sqeuclidean",
    splitting_cutoff=15**2,
    merging_cutoff=15**2,
)

In [4]:
regionprops_df.head()

,seg_id,y,x,frame
0,1,137.033613,96.336134,0
1,2,145.068966,178.988506,0
2,3,123.091743,203.266055,0
3,4,62.716216,141.290541,0
4,5,32.976744,124.348837,0


In [5]:
track_df, split_df, merge_df = lt.predict_dataframe(
    regionprops_df,
    coordinate_cols=coordinate_cols,
    frame_col=frame_col,
)

display(track_df.head())
display(split_df.head())
display(merge_df.head())

,seg_id,y,x,frame,tree_id,track_id
0,1,137.033613,96.336134,0,0,0
1,2,145.068966,178.988506,0,1,1
2,3,123.091743,203.266055,0,2,2
3,4,62.716216,141.290541,0,3,3
4,5,32.976744,124.348837,0,4,4


,parent_track_id,child_track_id
0,6,9
1,6,8
2,7,10
3,7,11


,parent_track_id,child_track_id


## Save to GEFF

In [6]:
geff_networks = data_conversion.dataframes_to_geff_networkx(
    track_df,
    split_df,
    merge_df,
    frame_col=frame_col,
)
for track_id in geff_networks.lineage_tree.nodes:
    geff_networks.lineage_tree.nodes[track_id]["length"] = len(
        list(
            filter(
                lambda n: n[1]["track_id"] == track_id,
                geff_networks.tree.nodes(data=True),
            )
        )
    )

In [7]:
print(geff_networks.tree.nodes[0])
print(geff_networks.lineage_tree.nodes[0])

{'frame': 0, 'seg_id': np.float64(1.0), 'y': np.float64(137.03361344537817), 'x': np.float64(96.33613445378151), 'tree_id': np.float64(0.0), 'track_id': np.float64(0.0)}
{'length': 10}


In [10]:
f = Path(tmp_path) / "tree.geff"
shutil.rmtree(f)
geff.write_nx(
    geff_networks.tree,
    f,
    axis_names=[frame_col] + coordinate_cols,
    axis_types=["time", "space", "space"],
    axis_units=["frame", "px", "px"],
)
geff.validate(f)

geff.write_nx(
    geff_networks.lineage_tree,
    f / "linage_tree.geff",
)
!ls {tmp_path}/"tree.geff"
!ls {f / "linage_tree.geff" / "nodes" / "props"}

edges  linage_tree.geff  nodes
length
